In [1]:
!pip install -q -U transformers datasets accelerate peft trl bitsandbytes huggingface_hub



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import torch
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
import os
from huggingface_hub import login

In [4]:



login()



Token has not been saved to git credential helper.


In [5]:
#model architeture
hf_dataset_name = "manohar3181/TarantinoBot-Finetuning-Dataset"
base_model_id = "mistralai/Mistral-7B-Instruct-v0.2"
output_dir = "/teamspace/studios/this_studio/TarantinoBot-Mistral-7B/checkpoints"
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)
model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
dataset = load_dataset(hf_dataset_name, split="train")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
def format_prompt(example):
    return f"<s>[INST] {example['instruction']} [/INST] {example['response']} </s>"


In [7]:
lora_config = LoraConfig(r=16, lora_alpha=32, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")
model = get_peft_model(prepare_model_for_kbit_training(model), lora_config)


In [12]:
training_args = TrainingArguments(
    output_dir="/teamspace/studios/this_studio/TarantinoBot-Mistral-7B/checkpoints",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    save_total_limit=3,
    learning_rate=5e-5,
    num_train_epochs=2,
    max_steps=-1,
    logging_steps=10,
    save_steps=10,
    fp16=True,
    weight_decay=0.01,
    max_grad_norm=0.3,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.01,
)

In [13]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=lora_config,
    formatting_func=format_prompt,
    args=training_args,
)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [15]:
trainer.train()

Step,Training Loss
10,2.774400
20,2.403900
30,2.285800
40,2.201100
50,2.102800
60,2.045400
70,2.039600
80,1.943800
90,1.930100
100,2.081100


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=208, training_loss=2.0404753501598654, metrics={'train_runtime': 1283.9484, 'train_samples_per_second': 0.648, 'train_steps_per_second': 0.162, 'total_flos': 2.553186627305472e+16, 'train_loss': 2.0404753501598654})